# Libraries

In [1]:
import os, os.path
import numpy as np
from pathlib import Path

import pandas as pd

In [3]:
notebooks_path = Path.cwd().parent
repo_path = notebooks_path.parent
os.chdir(str(notebooks_path))
#print current working directory
print(os.getcwd())

/home/ricardino/Documents/MAIA/tercer_semestre/CAD/Projecte/Machine Learning/notebooks


# Functions

## Retrive paths

In [11]:
classif = 'binary'
set_name = 'train'
class_name = 'nevus'

In [15]:
folder_path = str(repo_path) + f'/images/{classif}/{set_name}/{class_name}'
p = Path(folder_path).relative_to(str(repo_path)).glob('**/*')
files = [x for x in p if x.is_file()]
print(files)

[]


In [6]:
def get_paths(classif, set_name, class_name):
    """This function extracts the paths of all files in a folder, given the classifiation problem (binary, 3-class), the set name (train-val-test), and the class name
    (endpoint) of the diagnosis.
    It returns a list with all the paths in string.

    Args:
        classif (str): binary or 3-class
        set_name (str): train, val or test
        class_name (str): lesions name (nevus, etc.)
    """
    #path of the folder to check   
    folder_path = str(repo_path) + f'/images/{classif}/{set_name}/{class_name}'
    p = Path(folder_path).relative_to(str(repo_path)).glob('**/*')
    files = [x for x in p if x.is_file()]
    p = Path(folder_path).relative_to(str(repo_path)).glob('**/*')
    classes = [str(x.name)[:3] for x in p if x.is_file()]
    
    return files, classes


def info_dataframe(classif, set_name, class_name):
    """Returns df with Path, classification, set and class information

    Args:
        classif (_type_): _description_
        set_name (_type_): _description_
        class_name (_type_): _description_

    Returns:
        _type_: _description_
    """
    df = pd.DataFrame() #Save paths in df
    paths_list, classes = get_paths(classif, set_name, class_name) #Estract paths and classes
    paths = pd.DataFrame({ #save in frame
        'path': paths_list, 
        'class': classes
    })
    paths = paths.sort_values('path') #Sort paths by alphabetic order
    df = pd.concat([df,paths], ignore_index=True) #Concatenate with previous info
    df['classif'] = f'{classif}'
    df['set'] = f'{set_name}'
    
    #Assigning label
    if classif=='binary':
        df.loc[df['class'] =='nev', 'label'] = 1
        df.loc[df['class'] !='nev', 'label'] = 0
    if classif=='3_classes':
        df.loc[df['class'] =='bcc', 'label'] = 0
        df.loc[df['class'] =='mel', 'label'] = 1
        df.loc[df['class'] =='scc', 'label'] = 2
        
    
    return df

# 1. Getting Metadata

In [7]:
#Set classification task
classif_list = ['binary','3_classes']
set_list = ['train','val']

allInfo_df = pd.DataFrame() #Empty dataframe
for classif in classif_list:    
    for set_name in set_list:
        if classif=='binary':
            class_list = ['nevus', 'others']
        else:
            class_list = ['bcc', 'mel', 'scc']
        for class_name in class_list:

            currentInfo = info_dataframe(classif, set_name, class_name)
            allInfo_df = pd.concat([allInfo_df,currentInfo], ignore_index=True)

allInfo_df['ID'] = allInfo_df.index
# allInfo_df.to_csv('data/meta_info.csv', sep='\t', index=False)

ValueError: cannot set a frame with no defined index and a scalar

# Creating class to manipulate metadata

In [249]:
class path_label():
    def __init__(self, meta, classif, set_name) -> None:
        meta = meta.loc[meta['classif'] == classif] #Filter by classif
        meta = meta.loc[meta['set'] == set_name] #Filter by set
        self.paths = list(meta.path)
        self. labels = np.array(meta.label)